In [7]:
import numpy as np
from stl import mesh

# Load the STL file
stl_mesh = mesh.Mesh.from_file("/Users/ggito/repos/pinns/src/julia/f1_front_wing/files/ferrari_open_f1_2019_wing_v2_part.STL")

exception (False, "b'ha\\xcd\\xcc1c\\x04\\x02\\xc2b\\xcf\\x0cfa\\xcd\\xcc1c\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x00\\x80\\xbf\\x04\\x02\\xc2b\\xcf\\x0cfa\\xcd\\xcc1c\\xc7\\x96\\xc1b\\xef' should start with b'facet normal'")


In [8]:
# Calculate the area of each face
v0, v1, v2 = stl_mesh.v0, stl_mesh.v1, stl_mesh.v2
edge1 = v1 - v0
edge2 = v2 - v0
cross_product = np.cross(edge1, edge2)
areas = 0.5 * np.linalg.norm(cross_product, axis=1)
normals = cross_product / np.linalg.norm(cross_product, axis=1)[:, np.newaxis]

# Calculate the total surface area
total_area = np.sum(areas)

In [19]:
# Decide the number of points to sample
num_points_to_sample = 2000000

# Sample points and associate normals
sample_points = []
sample_normals = []
for i in range(len(stl_mesh)):
    num_points_in_face = int(areas[i] / total_area * num_points_to_sample)
    for _ in range(num_points_in_face):
        barycentric_coords = np.random.random(3)
        barycentric_coords /= barycentric_coords.sum()  # Ensure they sum to 1
        sample_point = (
            stl_mesh.v0[i] * barycentric_coords[0]
            + stl_mesh.v1[i] * barycentric_coords[1]
            + stl_mesh.v2[i] * barycentric_coords[2]
        )
        sample_points.append(sample_point)
        sample_normals.append(normals[i])


In [20]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

# Assuming sample_points and sample_normals are already defined

# Create a new matplotlib figure and axes.
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Create a Poly3DCollection object with the faces of the STL mesh.
poly_collection = Poly3DCollection(stl_mesh.vectors, facecolors='cyan', linewidths=0.05, edgecolors='r', alpha=0.1)
# ax.add_collection3d(poly_collection)

# Convert sample_points and sample_normals to numpy arrays if they are not already
sample_points = np.array(sample_points)
sample_normals = np.array(sample_normals)

# Swap Y and Z axes for all sampled points and normals (if needed)
sample_points = np.column_stack((sample_points[:, 0], sample_points[:, 2], sample_points[:, 1]))
sample_normals = np.column_stack((sample_normals[:, 0], sample_normals[:, 2], sample_normals[:, 1]))

# Scale the normals for visibility
normal_scale = 2  # Adjust this scale factor as needed
sample_normals_scaled = sample_normals * normal_scale

# Plot sampled points
ax.scatter(sample_points[:, 0], sample_points[:, 1], sample_points[:, 2], c='b', s=0.0005)

# Plot normal vectors
ax.quiver(
    sample_points[:, 0], sample_points[:, 1], sample_points[:, 2],
    sample_normals_scaled[:, 0], sample_normals_scaled[:, 1], sample_normals_scaled[:, 2],
    color='green', length=0.1, normalize=True
)

# Set axis labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Show plot
plt.show()

In [ ]:
import csv

# Specify the file name
points_filename = 'front_wing.csv'
norms_filename = 'front_wing_norms.csv'

# Open the file in write mode
with open(points_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header
    writer.writerow(['x', 'y', 'z'])
    
    # Write each point's coordinates as a row in the file
    for point in sample_points:
        writer.writerow(point)

with open(norms_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header
    writer.writerow(['x', 'y', 'z'])
    
    # Write each point's coordinates as a row in the file
    for point in sample_normals:
        writer.writerow(point)

In [12]:
min(sample_points[1])

12.333965